In [5]:
from pathlib import Path
from warnings import filterwarnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import requests
from io import StringIO
import rdkit
from sklearn import svm, metrics, clone
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import auc, accuracy_score, recall_score
from sklearn.metrics import roc_curve, roc_auc_score
from rdkit import Chem
from rdkit.Chem import MACCSkeys
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import rdmolops
from tqdm import tqdm_notebook
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from rdkit.Chem import SaltRemover
from IPython.display import SVG 
from rdkit.Chem import rdDepictor  
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit import RDLogger 
import json
import pickle
import os
from pathlib import Path
import random

RDLogger.DisableLog('rdApp.*')
import warnings
warnings.filterwarnings('ignore')
SEED = 22

random.seed(22)
os.environ["PYTHONHASHSEED"] = str(22)
np.random.seed(22)


HERE = Path.cwd()
s2m = lambda s: Chem.MolFromSmiles(s)
m2s = lambda m: Chem.MolToSmiles(m)
cancolise = lambda s: m2s(s2m(s)) 
get_dict_query = lambda q : data[str(int(q))]
get_maccs = lambda smiles: np.array(MACCSkeys.GenMACCSKeys(s2m(smiles)))
get_mrgn2 = lambda smiles: np.array(GetMorganFingerprintAsBitVect(s2m(smiles), 2, nBits=2048))
get_mrgn3 = lambda smiles: np.array(GetMorganFingerprintAsBitVect(s2m(smiles), 3, nBits=2048))

with open('id2name_byhand.json','r') as fp:
    id2name = json.load(fp)

In [9]:
len(set(id2name.values()))

452

In [ ]:
model_name = f"models/unbalance/{target}/{target}_{model}"


In [36]:
get_mrgn3_df('CCCCCCCCC')

array([0, 0, 0, ..., 0, 0, 0])

In [41]:
target= 'P19784'
target_items =[]
DATA = HERE / f"{target}"

# Getting Data
df = df[['id','smiles','active']]
data_count = {'Active':int(df.active.sum()),'Inactive':(len(df) - int(df.active.sum()))}
target_items.append(data_count)

#Dict for saving model information
target_only_dict = {}

#Getting Descriptor
df["fp_maccs"] = df["smiles"].apply(get_maccs) # MACCS descriptor
df["fp_mrgn"] = df["smiles"].apply(get_mrgn2)  # MORGAN OR ECFP2 descriptor  
fingerprint_maccs = df.fp_maccs.tolist()
fingerprint_mrgn = df.fp_mrgn.tolist()

#label
label = df.active.tolist()



for fingerprint in ['maccs','morgan2']:
    models = []
    if fingerprint == 'maccs':
        model_dict = traing_validate_plot_model(DATA,fingerprint, target,fingerprint_maccs, label)
    if fingerprint == 'morgan2':
        model_dict = traing_validate_plot_model(DATA,fingerprint, target,fingerprint_mrgn, label)
        

    target_items.append(model_dict)
target_only_dict[target] = target_items

with open(f'{DATA}/target_{target}.json', 'w') as fp:
    json.dump(target_only_dict, fp)


In [ ]:
for target in tqdm_notebook(targets):
    for model in models:
        try:
            # Absolute path of a file
            old_name = f"/home/melo/Documents/Pub'/models/unbalance/{target}/{model}"
            new_name = f"/home/melo/Documents/Pub'/models/unbalance/{target}/{target}_{model}"

            # Renaming the file
            os.rename(old_name, new_name)
        except:
            continue